In [ ]:
import numpy as np
import pandas as pd
import goodreads_api_client as gr
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import requests

from urllib.request import urlopen
import re

In [30]:
book.keys()

odict_keys(['id', 'title', 'isbn', 'isbn13', 'asin', 'kindle_asin', 'marketplace_id', 'country_code', 'image_url', 'small_image_url', 'publication_year', 'publication_month', 'publication_day', 'publisher', 'language_code', 'is_ebook', 'description', 'work', 'average_rating', 'num_pages', 'format', 'edition_information', 'ratings_count', 'text_reviews_count', 'url', 'link', 'authors', 'reviews_widget', 'popular_shelves', 'book_links', 'buy_links', 'series_works', 'similar_books'])

### Getting popular science isbns

In [294]:
h = {'Authorization': '44538_7390dbc4419639e04527db4ebb786883'}
resp = req.get("https://api2.isbndb.com/book/9781934759486", headers=h)
print(resp.json())

{'book': {'publisher': 'Robert Reed Publishers', 'synopsys': "Math Jokes 4 Mathy Folks Is An Absolute Gem...---jim Rubillo Professor Emeritus, Bucks County Community College, Newtown, Pa The Jokes In This Book Are Well-chosen And Cover A Wide Spectrum, From Jokes For Kids To Jokes For Math Majors, From Corny To Thought-provoking---art Benjamin Professor And Mathemagician, Harvey Mudd College, Claremont, Ca This Is A Book That Every Math Teacher From Elementary School Through College Should Have In Their Classroom Library. Who Said Math Can't Be Funny?---victoria Miles, Middle Grades Math Teacher, Weymouth, Ma Patrick Vennebush Has Put Together The Most Comprehensive Set Of Mathematical Jokes I Have Ever Seen...if You Like Math And You Like Jokes---or If You Need A Joke To Liven Up An Otherwise Dull And Boring Lecture---then You Need To Buy This Book.---guy Brandenburg, Retired Teacher, Washington, Dc Math Nerds And Punsters Rejoice! This Is The Book You've Been Waiting For---your Perfe

In [315]:
n = 201
resp_list = resp_list[:201]
while True:
    try:
        if n == 401:
            break
        resp = req.get(f"https://api2.isbndb.com/subject/popular%20science?page={n}&pageSize=50", headers=h)
        print(n)
        n += 1
        resp_list.append(resp)
    except:
        print("No more books")
        break
        
len(resp_list)

201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400


401

In [314]:
books = []
f = 0
for i, resp in enumerate(resp_list):
    try:
        books.extend(resp.json()['books'])
    except:
        print(f'Filed request index: {i}')
        f +=1
        if f == 5:
            break
    
len(books)

Filed request index: 200
Filed request index: 201
Filed request index: 202
Filed request index: 203
Filed request index: 204


10000

In [317]:
resp.json()['books']

KeyError: 'books'

### Books dataframe

In [318]:
pbooks_df = pd.DataFrame(books)
pbooks_df.to_csv('../data/popular_science_books.csv')

In [37]:
pbooks_df = pd.read_csv('../data/popular_science_books.csv')

### Getting reviews by isbn

In [2]:
import src.helper_functions as hf
from importlib import reload  

# while True:
#     # Do some things.
#     if is_changed(foo):
#         foo = reload(foo)

hf = reload(hf)

In [ ]:
pbooks_df = pd.read_csv('../data/popular_science_books.csv')
client = gr.Client(developer_key='fGkJVZsWA7At7eDPqUec4A')
isbns = list(pbooks_df['isbn'].astype(str))

def get_divs(isbn):
    link = hf.get_the_link(isbn)
    pages = hf.get_all_pages(link)
    divs = hf.div_reviews_all_pages(pages)
    print(len(divs))
    return divs

def user_rate_review(div, isbn):
    rating = hf.get_the_rating(div)
    review, full_name, link = hf.get_review(div)
    return {'isbn':isbn,'user_name' : full_name, "user_page":link, "ranking":rating, "review":review}

user_reviews = []
for isbn in isbns:
    try:
        divs = get_divs(isbn)
        print(len(divs))
        if len(divs) > 0:
            for div in divs:
                book_rating_review = user_rate_review(div)
                user_reviews.append(book_rating_review)
    except:
        print(f'Url not found for isbn: {isbn}')
        continue

Url not found for isbn: 1125642823


/Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py:10: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 10 of the file /Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs(widget, 'html')
/Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py:43: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but i

No more reviews


/Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py:28: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 28 of the file /Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs(r.content, "html")


0
0
No more reviews
0
0
No more reviews
0
0
No more reviews
3
3
Url not found for isbn: 1616284900
Url not found for isbn: 1616289678
Url not found for isbn: 1556540183
Url not found for isbn: 0943822858
Url not found for isbn: 1681881306
Url not found for isbn: 1681881284
Url not found for isbn: 2755613726
No more reviews
0
0
No more reviews
0
0
No more reviews
0
0
Url not found for isbn: 1681881292
Url not found for isbn: 1117850552
No more reviews
1
1
Url not found for isbn: 1616285311
No more reviews
1
1
Url not found for isbn: 0696111071
No more reviews
0
0
Url not found for isbn: 1258226618
No more reviews
0
0
Url not found for isbn: 1258227193
No more reviews
0
0
Url not found for isbn: 1258171422
Url not found for isbn: 0060133864
Url not found for isbn: 0486139379
No more reviews
0
0
Url not found for isbn: 1556540574
No more reviews
0
0
Url not found for isbn: 1258066599
Url not found for isbn: 1258457296
No more reviews
0
0
No more reviews
0
0
Url not found for isbn: 1258461

In [ ]:
df_user_review = pd.DataFrame(arr_user_review, columns=['ISBN13', 'uname', 'ulink', 'urating', 'ureview'])